# Police agencies

In [1]:
import json
import pandas as pd
import altair as alt
import altair_latimes as lat
pd.options.display.max_columns = 50
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.float_format = '{:,.0f}'.format 

In [2]:
df = pd.read_fwf('data/input/police/pe-2108.txt', \
                 colspecs=([0, 1],[1,3],[3,10],[10,12],[12,13],[13,15],[15,20],[20,23],\
                            [23,32],[32,56],[56,62],[62, 67],[67,72],[72, 77],[77,82],[82,87],\
                            [87,92],[92,97],[97,100],[100,103]), header=0,\
                names=['id', 'state_code', 'ori_code', 'group', 'division', 'year', 'sequence_num', 'msa', 'pop', 'name', 'state_name',\
                      'male_officers','male_civilians','male_total','female_officers','female_civilians','female_total',\
                       'total','officer_rate','employee_rate'])

In [3]:
group_codes = {
    '0':'Possessions (Puerto Rico, Guam, Canal Zone, Virgin Islands, and American Samoa)',
    '1':'All cities 250,000 or over:',
    '1A':'Cities 1,000,000 or over',
    '1B':'Cities from 500,000 thru 999,999 1C= Cities from 250,000 thru 499,999',
    '2':'Cities from 100,000 thru 249,000',
    '3':'Cities from 50,000 thru 99,000',
    '4':'Cities from 25,000 thru 49,999',
    '5':'Cities from 10,000 thru 24,999',
    '6':'Cities from 2,500 thru 9,999',
    '7':'Cities under 2,500',
    '8':'Non-MSA Counties:',
    '8A':'Non-MSA counties 100,000 or over',
    '8B':'Non-MSA counties from 25,000 thru 99,999 8C= Non-MSA counties from 10,000 thru 24,999 8D= Non-MSA counties under 10,000',
    '8E':'Non-MSA State Police',
    '9':'MSA Counties',
    '9A':'MSA counties 100,000 or over',
    '9B':'MSA counties from 25,000 thru 99,999 9C= MSA counties from 10,000 thru 24,999 9D= MSA counties under 10,000',
    '9E':'MSA State Police'}

In [4]:
group_abstract = {
    '0':'Possessions',
    '1':'City',
    '1A':'City',
    '1B':'City',
    '2':'City',
    '3':'City',
    '4':'City',
    '5':'City',
    '6':'City',
    '7':'City',
    '8':'County',
    '8A':'County',
    '8B':'County',
    '8E':'State Police',
    '9':'County',
    '9A':'County',
    '9B':'County',
    '9D':'County',
    '9E':'State Police'}

#### ORI codes from National Archive of Criminal Justice Data

In [5]:
# https://www.icpsr.umich.edu/icpsrweb/NACJD/studies/35158

In [6]:
ori = pd.read_csv('data/input/police/ICPSR_35158/35158-0001-Data.tsv', sep='\t',\
                  dtype={ 'FIPS':str, 'FPLACE':str, 'FIPS_ST':str, 'FIPS_COUNTY':str })

In [7]:
ori_slim = ori[(ori['LG_POPULATION'] != 888888888) & (ori['LG_POPULATION'] != 999999999)][['FIPS_ST', 'FIPS_COUNTY', 'FIPS','FPLACE', 'ORI7', 'NAME', \
                'STATENAME', 'COUNTYNAME', 'AGCYTYPE', \
                'LG_NAME', 'LG_POPULATION']]

In [8]:
ori_slim.columns = ori_slim.columns.str.strip().str.lower().str.replace(' ','_')\
    .str.replace('(', '').str.replace(')', '').str.replace('/','_')

In [9]:
ori_slim.head(10)

,fips_st,fips_county,fips,fplace,ori7,name,statename,countyname,agcytype,lg_name,lg_population
0,01,001,01001,3220,AL00402,AUTAUGAVILLE POLICE DEPARTMENT,ALABAMA,AUTAUGA,0,AUTAUGAVILLE TOWN,870
1,01,001,01001,62328,AL00401,PRATTVILLE POLICE DEPARTMENT,ALABAMA,AUTAUGA,0,PRATTVILLE CITY,33960
3,01,001,01001,99001,AL00400,AUTAUGA COUNTY SHERIFF'S OFFICE,ALABAMA,AUTAUGA,1,AUTAUGA COUNTY,54571
5,01,003,01003,4660,AL00501,BAY MINETTE POLICE DEPARTMENT,ALABAMA,BALDWIN,0,BAY MINETTE CITY,8044
6,01,003,01003,19648,AL00508,DAPHNE POLICE DEPARTMENT,ALABAMA,BALDWIN,0,DAPHNE CITY,21570
7,01,003,01003,23320,AL00510,ELBERTA POLICE DEPARTMENT,ALABAMA,BALDWIN,0,ELBERTA TOWN,1498
8,01,003,01003,25240,AL00502,FAIRHOPE POLICE DEPARTMENT,ALABAMA,BALDWIN,0,FAIRHOPE CITY,15326
9,01,003,01003,26992,AL00503,FOLEY POLICE DEPARTMENT,ALABAMA,BALDWIN,0,FOLEY CITY,14618
10,01,003,01003,32272,AL00504,GULF SHORES POLICE DEPARTMENT,ALABAMA,BALDWIN,0,GULF SHORES CITY,9741
11,01,003,01003,44608,AL00506,LOXLEY POLICE DEPARTMENT,ALABAMA,BALDWIN,0,LOXLEY TOWN,1632


In [10]:
merge = df.merge(ori_slim, left_on='ori_code', right_on='ori7').drop(['name_x'], axis=1).rename(columns={'name_y':'agency_name'})

In [11]:
merge['type_desc'] = merge.group.map(group_codes)
merge['type'] = merge.group.map(group_abstract).str.upper()

In [12]:
merge_slim = pd.DataFrame(merge[['agency_name', 'statename', 'countyname', 'lg_name', 'lg_population', 'fips', \
              'fplace', 'type']]).rename(columns={ 'statename':'state', 'countyname':'county', 'lg_name':'place', \
                           'lg_population':'pop', 'male_officers':'male', 'female_officers':'female', 'officer_rate':'rate_10k'})

#### Which large agencies have the highest officer rate?

In [13]:
merge_slim[merge_slim['pop']>100000].head(10)

,agency_name,state,county,place,pop,fips,fplace,type
21,ANCHORAGE PD,ALASKA,ANCHORAGE MUNICIPALITY,ANCHORAGE MUNICIPALITY,291826,02020,3000,CITY
38,JEFFERSON COUNTY SHERIFF'S OFFICE,ALABAMA,JEFFERSON,JEFFERSON COUNTY,658466,01073,99073,COUNTY
40,BIRMINGHAM POLICE DEPARTMENT,ALABAMA,JEFFERSON,BIRMINGHAM CITY,212237,01073,7000,CITY
66,MOBILE COUNTY SHERIFF'S OFFICE,ALABAMA,MOBILE,MOBILE COUNTY,412992,01097,99097,COUNTY
67,MOBILE POLICE DEPARTMENT,ALABAMA,MOBILE,MOBILE CITY,195111,01097,50000,CITY
77,MONTGOMERY COUNTY SHERIFF'S OFFICE,ALABAMA,MONTGOMERY,MONTGOMERY COUNTY,229363,01101,99101,COUNTY
78,MONTGOMERY POLICE DEPARTMENT,ALABAMA,MONTGOMERY,MONTGOMERY CITY,205764,01101,51000,CITY
82,BALDWIN COUNTY SHERIFF'S OFFICE,ALABAMA,BALDWIN,BALDWIN COUNTY,182265,01003,99003,COUNTY
118,CALHOUN COUNTY SHERIFF'S OFFICE,ALABAMA,CALHOUN,CALHOUN COUNTY,118572,01015,99015,COUNTY
230,ETOWAH COUNTY SHERIFF'S OFFICE,ALABAMA,ETOWAH,ETOWAH COUNTY,104430,01055,99055,COUNTY


#### Just California

In [14]:
df_ca = merge_slim[merge_slim['state'] == 'CALIFORNIA']

In [15]:
df_ca.sort_values(by='pop', ascending=False).head(20)

,agency_name,state,county,place,pop,fips,fplace,type
945,LOS ANGELES COUNTY SHERIFF'S OFFICE,CALIFORNIA,LOS ANGELES,LOS ANGELES COUNTY,9818605,06037,99037,COUNTY
995,LOS ANGELES POLICE DEPARTMENT,CALIFORNIA,LOS ANGELES,LOS ANGELES CITY,3792621,06037,44000,CITY
1197,SAN DIEGO COUNTY SHERIFF'S OFFICE,CALIFORNIA,SAN DIEGO,SAN DIEGO COUNTY,3095313,06073,99073,COUNTY
1086,ORANGE CNTY SHERIFFS DEPT,CALIFORNIA,ORANGE,ORANGE COUNTY,3010232,06059,99059,COUNTY
1158,JURUPA VALLEY,CALIFORNIA,RIVERSIDE COUNTY,RIVERSIDE COUNTY,2189641,06065,99065,CITY
1157,EASTVALE,CALIFORNIA,RIVERSIDE COUNTY,RIVERSIDE COUNTY,2189641,06065,99065,CITY
1129,RIVERSIDE COUNTY SHERIFF'S OFFICE,CALIFORNIA,RIVERSIDE,RIVERSIDE COUNTY,2189641,06065,99065,COUNTY
1171,SAN BERNARDINO COUNTY SHERIFF'S OFFICE,CALIFORNIA,SAN BERNARDINO,SAN BERNARDINO COUNTY,2035210,06071,99071,COUNTY
1263,SANTA CLARA COUNTY SHERIFF'S OFFICE,CALIFORNIA,SANTA CLARA,SANTA CLARA COUNTY,1781642,06085,99085,COUNTY
828,ALAMEDA COUNTY SHERIFF'S OFFICE,CALIFORNIA,ALAMEDA,ALAMEDA COUNTY,1510271,06001,99001,COUNTY


In [16]:
merge_slim.to_csv('data/input/departments.csv')